In [1]:
import pandas as pd
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

pd.set_option('display.max.colwidth',30)
pd.options.display.max_columns = 100

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time
from bs4 import BeautifulSoup
import re

# Youtuber 6 - 소소황
- 213개

In [3]:
# 원하는 유튜버 '동영상'에 들어가기 
driver = webdriver.Chrome('chromedriver.exe')
driver.get('https://www.youtube.com/channel/UCWrh9haWsTEAQF9UEg6AEFQ/videos')
body = driver.find_element_by_tag_name('body')
driver.maximize_window()
time.sleep(1.5)#브라우저 로딩시간기다려야함

In [4]:
# 유튜브는 스크롤해야 전체를 볼 수 있음 
endk = 70
while endk:
    driver.find_element_by_tag_name('body').send_keys(Keys.END)
    time.sleep(0.3)
    endk -= 1   
    
start_page = driver.page_source 
soup = BeautifulSoup(start_page,'lxml') # 전체 영상 목록의 page-source 이니 soup으로

# 각 영상 페이지의 url 모음
video_list = soup.find_all('ytd-grid-video-renderer',{'class':'style-scope ytd-grid-renderer'})

SS_url= []
base_url = 'http://www.youtube.com'
for i in range(len(video_list)):
    url = base_url+video_list[i].find('a',{'id':'thumbnail'})['href']
    SS_url.append(url)

In [5]:
len(SS_url)

213

In [20]:
SS = pd.DataFrame({'youtuber' :[],
                   'subscribers' : [],
                   'name':[],
#                    'video_url':[],
                   'upload_date':[],
                   'hits' :[],
                   'likes_num':[],
                   'dislikes_num':[]})

for i in range(0,len(SS_url)):

    try :# 오휴생겨서 try-except 추가
        youtuber = soup.find('title').text
        sub_num = soup.find('yt-formatted-string',{'id':'subscriber-count'}).text
        name = video_list[i].find('a',{'id':'video-title'}).text
    #     url = base_url+video_list[i].find('a',{'id':'thumbnail'})['href']


        # 각 동영상 페이지 들어가기 시작 
        start_url = SS_url[i]
        driver.get(start_url) 
        body = driver.find_element_by_tag_name('body')

        time.sleep(1.5)

        # 댓글까지 쭉 보려면 스크롤
        endk = 5 # 5번 내림
        while endk:
            driver.find_element_by_tag_name('body').send_keys(Keys.END)
            time.sleep(0.3) # 내리는 시간 간격 
            endk -= 1    
        page = driver.page_source 
        html = BeautifulSoup(page,'lxml') # 각 페이지의 page_source 이니 html로


        # 각 동영상 페이지 들어가서 얻을 수 있는 정보
        pre_upload_date = html.find('div',{'id':'date'})
        upload_date = pre_upload_date.find('yt-formatted-string', {'class':'style-scope ytd-video-primary-info-renderer'}).text   
        pre_hits = html.find('yt-view-count-renderer', {'class':'style-scope ytd-video-primary-info-renderer'})
        hits = pre_hits.find('span',{'class':'view-count style-scope yt-view-count-renderer'}).text.split()[1]
        likes_num = html.find('yt-formatted-string',{'id':'text','class':'style-scope ytd-toggle-button-renderer style-text','aria-label':re.compile('좋아요')}).text+'개'
        dislikes_num = html.find('yt-formatted-string',{'id':'text','class':'style-scope ytd-toggle-button-renderer style-text','aria-label':re.compile('싫어요')}).text+'개'


        # 마지막에 모은 정보 합치기
        insert_data = pd.DataFrame({'youtuber' :[youtuber],
                                    'subscribers' : [sub_num],
                                    'name':[name],
    #                                 'video_url':[url],
                                    'upload_date':[upload_date], 
                                    'hits' : [hits],
                                    'likes_num':[likes_num],
                                    'dislikes_num':[dislikes_num]})

        SS = SS.append(insert_data)
        
    except AttributeError as e: 
        print(e)

In [22]:
SS

,youtuber,subscribers,name,upload_date,hits,likes_num,dislikes_num
0,소소황 Cook & Eat - YouTube,구독자 12.7만명,달걀 깨기만 하면 끝! 밥도둑 노른자장 만들기_...,2020. 3. 29.,"15,604회",198개,10개
0,소소황 Cook & Eat - YouTube,구독자 12.7만명,매콤 치즈불닭 만들기_ 깔끔하고 맛있게 매운 스...,2020. 3. 27.,"5,970회",187개,1개
0,소소황 Cook & Eat - YouTube,구독자 12.7만명,바삭바삭 감자채전 만들기_ 감자로 만드는 가장 ...,2020. 3. 25.,"42,673회",638개,10개
0,소소황 Cook & Eat - YouTube,구독자 12.7만명,밑반찬 3대장 [쪽파김무침] 맛있게 만드는 법_...,2020. 3. 23.,"7,764회",220개,1개
0,소소황 Cook & Eat - YouTube,구독자 12.7만명,닭강정집보다 맛있는 매콤달콤 양념 닭강정 만들기...,2020. 3. 21.,"14,570회",358개,5개
0,소소황 Cook & Eat - YouTube,구독자 12.7만명,매콤 쭈꾸미볶음 만들기_ 지금 먹어야 가장 쫄깃...,2020. 3. 20.,"13,425회",311개,5개
0,소소황 Cook & Eat - YouTube,구독자 12.7만명,1000번 저어 만드는 계란후라이 만들기,2020. 3. 18.,"10,755회",171개,3개
0,소소황 Cook & Eat - YouTube,구독자 12.7만명,마약김밥 만드는 법_ 새우깡 이후 이렇게 자꾸 ...,2020. 3. 17.,"189,433회",1.6천개,53개
0,소소황 Cook & Eat - YouTube,구독자 12.7만명,두부양념조림 맛있게 만드는 법_ 매콤짭쪼름 기가...,2020. 3. 15.,"25,629회",401개,7개
0,소소황 Cook & Eat - YouTube,구독자 12.7만명,닭볶음탕 만들기_ '이것' 한스푼이면 감칠맛이...,2020. 3. 13.,"47,710회",504개,21개


In [23]:
SS.to_csv('SS.csv', encoding='utf-8-sig')

# YouTuber 6 - 심방골주부  
- 367개

In [24]:
# 원하는 유튜버 '동영상'에 들어가기 
driver = webdriver.Chrome('chromedriver.exe')
driver.get('https://www.youtube.com/channel/UCMXFSqYCTcB4iIJLwUJaBrQ/videos')
body = driver.find_element_by_tag_name('body')
driver.maximize_window()
time.sleep(1.5)#브라우저 로딩시간기다려야함

In [25]:
# 유튜브는 스크롤해야 전체를 볼 수 있음 
endk = 110
while endk:
    driver.find_element_by_tag_name('body').send_keys(Keys.END)
    time.sleep(0.3)
    endk -= 1   
    
start_page = driver.page_source 
soup = BeautifulSoup(start_page,'lxml') # 전체 영상 목록의 page-source 이니 soup으로

# 각 영상 페이지의 url 모음
video_list = soup.find_all('ytd-grid-video-renderer',{'class':'style-scope ytd-grid-renderer'})

SB_url= []
base_url = 'http://www.youtube.com'
for i in range(len(video_list)):
    url = base_url+video_list[i].find('a',{'id':'thumbnail'})['href']
    SB_url.append(url)

In [28]:
SB = pd.DataFrame({'youtuber' :[],
                   'subscribers' : [],
                   'name':[],
#                    'video_url':[],
                   'upload_date':[],
                   'hits' :[],
                   'likes_num':[],
                   'dislikes_num':[]})

for i in range(0,len(SB_url)):
    
    try :# 오휴생겨서 try-except 추가
        youtuber = soup.find('title').text
        sub_num = soup.find('yt-formatted-string',{'id':'subscriber-count'}).text
        name = video_list[i].find('a',{'id':'video-title'}).text
    #     url = base_url+video_list[i].find('a',{'id':'thumbnail'})['href']


        # 각 동영상 페이지 들어가기 시작 
        start_url = SB_url[i]
        driver.get(start_url) 
        body = driver.find_element_by_tag_name('body')

        time.sleep(1.5)

        # 댓글까지 쭉 보려면 스크롤
        endk = 5 # 5번 내림
        while endk:
            driver.find_element_by_tag_name('body').send_keys(Keys.END)
            time.sleep(0.3) # 내리는 시간 간격 
            endk -= 1    
        page = driver.page_source 
        html = BeautifulSoup(page,'lxml') # 각 페이지의 page_source 이니 html로


        # 각 동영상 페이지 들어가서 얻을 수 있는 정보
        pre_upload_date = html.find('div',{'id':'date'})
        upload_date = pre_upload_date.find('yt-formatted-string', {'class':'style-scope ytd-video-primary-info-renderer'}).text   
        pre_hits = html.find('yt-view-count-renderer', {'class':'style-scope ytd-video-primary-info-renderer'})
        hits = pre_hits.find('span',{'class':'view-count style-scope yt-view-count-renderer'}).text.split()[1]
        likes_num = html.find('yt-formatted-string',{'id':'text','class':'style-scope ytd-toggle-button-renderer style-text','aria-label':re.compile('좋아요')}).text+'개'
        dislikes_num = html.find('yt-formatted-string',{'id':'text','class':'style-scope ytd-toggle-button-renderer style-text','aria-label':re.compile('싫어요')}).text+'개'
        
    
        # 마지막에 모은 정보 합치기
        insert_data = pd.DataFrame({'youtuber' :[youtuber],
                                    'subscribers' : [sub_num],
                                    'name':[name],
    #                                 'video_url':[url],
                                    'upload_date':[upload_date], 
                                    'hits' : [hits],
                                    'likes_num':[likes_num],
                                    'dislikes_num':[dislikes_num]})

        SB = SB.append(insert_data)
    
    except AttributeError as e: 
        print(e)

'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'


In [29]:
SB.to_csv('SB.csv', encoding='utf-8-sig')

In [36]:
SB

,youtuber,subscribers,name,upload_date,hits,likes_num,dislikes_num
0,심방골주부Korean Food Recipes -...,구독자 36.4만명,주(쭈)꾸미볶음 매운게 땡길땐 제철 쭈꾸미로 볶...,2020. 3. 30.,"1,216회",146개,6개
0,심방골주부Korean Food Recipes -...,구독자 36.4만명,해물파전 집에서도 쉽게 만드는 법 레시피심방골주부,2020. 3. 26.,"8,353회",362개,4개
0,심방골주부Korean Food Recipes -...,구독자 36.4만명,열무김치 맛있게 담는법 초보자도 하나하나 담을 ...,2020. 3. 24.,"25,680회",689개,22개
0,심방골주부Korean Food Recipes -...,구독자 36.4만명,미나리무침 향긋한 미나리의 향 가득한 손쉬운 나...,2020. 3. 21.,"17,286회",442개,10개
0,심방골주부Korean Food Recipes -...,구독자 36.4만명,찹쌀고추장 담그는법 처음부터 끝까지 전과정 레시...,2020. 3. 4.,"27,949회",590개,30개
0,심방골주부Korean Food Recipes -...,구독자 36.4만명,오이무침 새콤달콤한 맛과 쉽고 빠르게 만들 수 ...,2020. 3. 2.,"16,631회",536개,11개
0,심방골주부Korean Food Recipes -...,구독자 36.4만명,도라지생채(무침) 몸에 좋은 도라지로 면역력을 ...,2020. 2. 26.,"10,700회",388개,10개
0,심방골주부Korean Food Recipes -...,구독자 36.4만명,얼갈이배추 겉절이 산뜻하게 봄맞이 김치 만들어보...,2020. 2. 26.,"38,843회",765개,21개
0,심방골주부Korean Food Recipes -...,구독자 36.4만명,총각 김치볶음밥 간편하게 해먹을 수 있는 한끼식...,2020. 2. 24.,"14,369회",432개,7개
0,심방골주부Korean Food Recipes -...,구독자 36.4만명,심방골의 겨울(눈)풍경과 김치수제비,2020. 2. 21.,"53,369회",1.5천개,37개


# YouTuber 7 - 강쉪
- 504개

In [41]:
# 원하는 유튜버 '동영상'에 들어가기 
driver = webdriver.Chrome('chromedriver.exe')
driver.get('https://www.youtube.com/channel/UCrxSR4OGbJBVjYxp4xjWgZQ/videos')
body = driver.find_element_by_tag_name('body')
driver.maximize_window()
time.sleep(1.5)#브라우저 로딩시간기다려야함

In [42]:
# 유튜브는 스크롤해야 전체를 볼 수 있음 
endk = 200
while endk:
    driver.find_element_by_tag_name('body').send_keys(Keys.END)
    time.sleep(0.3)
    endk -= 1   
    
start_page = driver.page_source 
soup = BeautifulSoup(start_page,'lxml') # 전체 영상 목록의 page-source 이니 soup으로

# 각 영상 페이지의 url 모음
video_list = soup.find_all('ytd-grid-video-renderer',{'class':'style-scope ytd-grid-renderer'})

KC_url= []
base_url = 'http://www.youtube.com'
for i in range(len(video_list)):
    url = base_url+video_list[i].find('a',{'id':'thumbnail'})['href']
    KC_url.append(url)

In [43]:
len(KC_url)

504

In [44]:
KC = pd.DataFrame({'youtuber' :[],
                   'subscribers' : [],
                   'name':[],
#                    'video_url':[],
                   'upload_date':[],
                   'hits' :[],
                   'likes_num':[],
                   'dislikes_num':[]})

for i in range(0,len(KC_url)):
    
    try :# 오휴생겨서 try-except 추가
        youtuber = soup.find('title').text
        sub_num = soup.find('yt-formatted-string',{'id':'subscriber-count'}).text
        name = video_list[i].find('a',{'id':'video-title'}).text
    #     url = base_url+video_list[i].find('a',{'id':'thumbnail'})['href']


        # 각 동영상 페이지 들어가기 시작 
        start_url = KC_url[i]
        driver.get(start_url) 
        body = driver.find_element_by_tag_name('body')

        time.sleep(1.5)

        # 댓글까지 쭉 보려면 스크롤
        endk = 5 # 5번 내림
        while endk:
            driver.find_element_by_tag_name('body').send_keys(Keys.END)
            time.sleep(0.3) # 내리는 시간 간격 
            endk -= 1    
        page = driver.page_source 
        html = BeautifulSoup(page,'lxml') # 각 페이지의 page_source 이니 html로


        # 각 동영상 페이지 들어가서 얻을 수 있는 정보
        pre_upload_date = html.find('div',{'id':'date'})
        upload_date = pre_upload_date.find('yt-formatted-string', {'class':'style-scope ytd-video-primary-info-renderer'}).text   
        pre_hits = html.find('yt-view-count-renderer', {'class':'style-scope ytd-video-primary-info-renderer'})
        hits = pre_hits.find('span',{'class':'view-count style-scope yt-view-count-renderer'}).text.split()[1]
        likes_num = html.find('yt-formatted-string',{'id':'text','class':'style-scope ytd-toggle-button-renderer style-text','aria-label':re.compile('좋아요')}).text+'개'
        dislikes_num = html.find('yt-formatted-string',{'id':'text','class':'style-scope ytd-toggle-button-renderer style-text','aria-label':re.compile('싫어요')}).text+'개'
        
    
        # 마지막에 모은 정보 합치기
        insert_data = pd.DataFrame({'youtuber' :[youtuber],
                                    'subscribers' : [sub_num],
                                    'name':[name],
    #                                 'video_url':[url],
                                    'upload_date':[upload_date], 
                                    'hits' : [hits],
                                    'likes_num':[likes_num],
                                    'dislikes_num':[dislikes_num]})

        KC = KC.append(insert_data)
    
    except AttributeError as e: 
        print(e)

In [46]:
KC.to_csv('KC.csv', encoding='utf-8-sig')

In [45]:
KC

,youtuber,subscribers,name,upload_date,hits,likes_num,dislikes_num
0,강쉪 - YouTube,구독자 19.5만명,간단하고 맛있는 쪽파 요리 3가지~ 3 kind...,2020. 3. 27.,"243,140회",2.6천개,63개
0,강쉪 - YouTube,구독자 19.5만명,간단하고 맛있는 느타리 버섯 3가지 요리~ 3 ...,2020. 3. 25.,"236,677회",3.7천개,80개
0,강쉪 - YouTube,구독자 19.5만명,간단하고 맛있는 무 김치 3가지~ 3 kind...,2020. 3. 22.,"141,622회",1.8천개,30개
0,강쉪 - YouTube,구독자 19.5만명,계란으로 할수있는 간단하고 맛있는 3가지 요리~...,2020. 3. 19.,"262,820회",3천개,86개
0,강쉪 - YouTube,구독자 19.5만명,간단하고 맛있는 등갈비 3가지 요리~ 3 kin...,2020. 3. 15.,"326,359회",3.2천개,79개
0,강쉪 - YouTube,구독자 19.5만명,간단하고 맛있는 오징어로 할수있는 3가지 요리~...,2020. 3. 13.,"152,409회",1.9천개,47개
0,강쉪 - YouTube,구독자 19.5만명,간단하고 맛있는 미트볼 요리 3가지~ 3 kin...,2020. 3. 10.,"11,274회",234개,6개
0,강쉪 - YouTube,구독자 19.5만명,간단하고 맛있는 오이 요리 3가지~ 3 kin...,2020. 3. 5.,"460,869회",5천개,137개
0,강쉪 - YouTube,구독자 19.5만명,간단하고 맛있는 대패삼겹살 3가지요리~ 3 ki...,2020. 3. 3.,"366,077회",2.7천개,87개
0,강쉪 - YouTube,구독자 19.5만명,간단하고 맛있는 팽이버섯 요리 3가지~ 3 ki...,2020. 2. 28.,"992,395회",9천개,399개
